# Basic Linear Algebra in PyTorch

Linear algebra forms the computational backbone of modern machine learning. In this notebook, we'll explore how PyTorch implements these operations efficiently, using temperature data as our running example.

Three key ideas drive PyTorch's design:
1. Tensors extend vectors and matrices to arbitrary dimensions, enabling batch processing
2. Memory layout and broadcasting optimize computation through cache-friendly operations
3. SVD reveals low-dimensional structure in high-dimensional data

These concepts power everything from computer vision to natural language processing, but we'll build intuition through a simpler domain: temperature analysis.

In [1]:
# Essential imports
import torch
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Configure matplotlib for notebook display
%matplotlib inline

# Set default tensor type for consistent precision
torch.set_default_dtype(torch.float32)

# Utility function for consistent printing
def print_tensor(name, x):
    """Print tensor with name and shape."""
    print(f"{name}: shape={x.shape}")
    print(x)

## Vectors and Tensors: The Foundation
    
Vectors represent sequences of measurements. While mathematical vectors are abstract, computational vectors must handle real-world constraints like memory layout and numerical precision:

In [2]:
# Temperature readings (Celsius)
readings = torch.tensor([22.5, 23.1, 21.8])  # Morning, noon, night
print(f"Readings: {readings}")
print(f"Shape: {readings.shape}")
print(f"Data type: {readings.dtype}")  # PyTorch chooses optimal precision

Readings: tensor([22.5000, 23.1000, 21.8000])
Shape: torch.Size([3])
Data type: torch.float32


PyTorch implements vector operations through SIMD (Single Instruction Multiple Data) parallelism:

In [3]:
# Compare two days
morning = torch.tensor([22.5, 23.1, 21.8])  # Yesterday
evening = torch.tensor([21.0, 22.5, 20.9])  # Today

# Vector operations - each element processed in parallel
total = morning + evening
print(f"Sum: {total}")

alpha = 0.5  # Averaging weight
weighted = alpha * morning  # Vectorized scalar multiplication
print(f"Weighted: {weighted}")

Sum: tensor([43.5000, 45.6000, 42.7000])
Weighted: tensor([11.2500, 11.5500, 10.9000])


### Creating Tensors
    
PyTorch provides multiple tensor creation methods, each optimized for different use cases:

In [4]:
# Vector creation methods
temps = torch.tensor([22.5, 23.1, 21.8])     # From data - copies input
zeros = torch.zeros(3)                        # Initialized - contiguous memory
weekly = torch.randn(7, 3)                    # Random normal - vectorized generation

print(f"Vector shape: {temps.shape}")
print(f"Matrix shape: {weekly.shape}")
print(f"Memory layout: {weekly.stride()}")    # Shows how data is stored

Vector shape: torch.Size([3])
Matrix shape: torch.Size([7, 3])
Memory layout: (3, 1)


### Vector Operations
    
Key operations combine computational efficiency with mathematical elegance:

In [5]:
# Analyzing temperature patterns
day1 = torch.tensor([22.5, 23.1, 21.8])  # Warmer day
day2 = torch.tensor([21.0, 22.5, 20.9])  # Cooler day

# Pattern similarity through optimized BLAS operations
similarity = torch.dot(day1, day2)        # Uses hardware-optimized dot product

# L2 norms computed efficiently through BLAS
mag1 = torch.norm(day1, p=2)              # Stable computation via scaling
mag2 = torch.norm(day2, p=2)              # Explicit L2 norm

# Cosine similarity - numerically stable implementation
cos_theta = similarity / (mag1 * mag2)

print(f"Similarity: {similarity:.1f}")
print(f"Day 1 magnitude: {mag1:.1f}")
print(f"Day 2 magnitude: {mag2:.1f}")
print(f"Pattern similarity: {cos_theta:.3f}")

Similarity: 1447.9
Day 1 magnitude: 38.9
Day 2 magnitude: 37.2
Pattern similarity: 1.000


The high cosine value (near 1) reveals that temperature patterns remain consistent even as absolute values shift. This stability reflects fundamental physical constraints on daily temperature variations.

### Quick Check: Vector Operations
    
Compute the average deviation from mean - a key statistical measure:

In [6]:
readings = torch.tensor([22.5, 23.1, 21.8])
mean = readings.mean()                                    # Stable one-pass algorithm
deviations = readings - mean                             # Vectorized subtraction
magnitude = torch.sqrt(torch.dot(deviations, deviations))  # Numerically stable norm
print(f"Average deviation: {magnitude/3:.4f}")  # Should be around 0.31

Average deviation: 0.3067


## Matrix Operations

Matrices enable batch processing of multiple measurements. PyTorch optimizes matrix operations through:
1. Cache-friendly memory layouts
2. Hardware-accelerated BLAS routines
3. Automatic operation fusion

In [7]:
# One week of temperature readings (7 days × 3 times per day)
week_temps = torch.tensor([
    [22.5, 23.1, 21.8],  # Monday
    [21.0, 22.5, 20.9],  # Tuesday
    [23.1, 24.0, 22.8],  # Wednesday
    [22.8, 23.5, 21.9],  # Thursday
    [21.5, 22.8, 21.2],  # Friday
    [20.9, 21.8, 20.5],  # Saturday
    [21.2, 22.0, 20.8]   # Sunday
])
print(f"Shape: {week_temps.shape}")          # Logical structure
print(f"Strides: {week_temps.stride()}")     # Physical memory layout
print(f"Total elements: {week_temps.numel()}") # Number of elements

Shape: torch.Size([7, 3])
Strides: (3, 1)
Total elements: 21


### Basic Matrix Operations
    
PyTorch fuses multiple operations for efficiency:

In [8]:
# Compare weeks with fused operations
last_week = torch.tensor([
    [21.5, 22.1, 20.8],  # Morning, noon, night
    [20.0, 21.5, 19.9],
    [22.1, 23.0, 21.8]
])

this_week = torch.tensor([
    [22.5, 23.1, 21.8],
    [21.0, 22.5, 20.9],
    [23.1, 24.0, 22.8]
])

# Temperature changes - single fused operation
temp_change = this_week - last_week  # No temporary storage needed
print("Temperature changes:")
print(temp_change)

# Efficient reduction along specified axis
daily_means = this_week.mean(dim=0)  # Uses stable online algorithm
print("\nAverage temperatures:")
print(daily_means)  # Morning, Noon, Night averages

Temperature changes:
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

Average temperatures:
tensor([22.2000, 23.2000, 21.8333])


### Matrix Multiplication
    
Matrix multiplication leverages highly optimized BLAS (Basic Linear Algebra Subprograms):

![Matrix Multiplication](figures/matrix_multiply.png)

The operation is optimized through:
1. Cache blocking for memory efficiency
2. SIMD vectorization for parallel computation
3. Multi-threading for large matrices

In [9]:
# Temperature readings and importance weights
temps = torch.tensor([
    [22.5, 23.1, 21.8],  # Day 1: morning, noon, night
    [21.0, 22.5, 20.9],  # Day 2
    [23.1, 24.0, 22.8]   # Day 3
], dtype=torch.float32)  # Specify precision for BLAS

weights = torch.tensor([0.5, 0.3, 0.2])  # Recent days matter more

# Efficient matrix-vector multiply using BLAS
weighted_means = torch.mv(temps, weights)  # Optimized for matrix-vector product
print("Weighted averages per time:")
print(weighted_means)

Weighted averages per time:
tensor([22.5400, 21.4300, 23.3100])


### Broadcasting
    
Broadcasting enables efficient operations between different shapes without memory copies:
    
![Broadcasting](figures/broadcasting.png)
    
The rules ensure memory efficiency while maintaining mathematical clarity.

In [10]:
# Temperature readings across days with efficient broadcasting
day_temps = torch.tensor([
    [22.5, 23.1, 21.8],  # Day 1: morning, noon, night
    [21.0, 22.5, 20.9],  # Day 2
    [23.1, 24.0, 22.8]   # Day 3
])

# Sensor calibration factors (per time of day)
calibration = torch.tensor([1.02, 0.98, 1.01])

# Broadcasting: implicit expansion without memory allocation
calibrated = day_temps * calibration  # Efficient in-place operation
print("Original vs Calibrated (first day):")
print(day_temps[0])      # Before calibration
print(calibrated[0])     # After calibration
print(f"Memory efficiency: input elements = {day_temps.numel()}, output elements = {calibrated.numel()}")

Original vs Calibrated (first day):
tensor([22.5000, 23.1000, 21.8000])
tensor([22.9500, 22.6380, 22.0180])
Memory efficiency: input elements = 9, output elements = 9


### Memory Layout
    
Understanding memory layout is crucial for performance:
    
![Memory Layout](figures/memory_layout.png)
    
Row-major storage affects operation speed:

In [11]:
# Memory access patterns affect performance
day_readings = week_temps[0]        # Fast: contiguous memory
morning_temps = week_temps[:, 0]    # Slower: strided access

# Demonstrate layout impact
print("Row access stride:", week_temps[0].stride())      # Small stride
print("Column access stride:", week_temps[:, 0].stride()) # Large stride
print("Memory layout:", week_temps.is_contiguous())      # Check if contiguous

Row access stride: (1,)
Column access stride: (3,)
Memory layout: True


### Quick Check: Matrix Operations
    
Combine broadcasting and matrix multiplication efficiently:

In [12]:
# Temperature data with specified memory layout
temps = torch.tensor([
    [22.5, 23.1, 21.8],
    [21.0, 22.5, 20.9]
], dtype=torch.float32)  # Ensure BLAS compatibility

calibration = torch.tensor([1.02, 0.98, 1.01])  # Per-time calibration
weights = torch.tensor([0.7, 0.3])              # Weights for each day

# Fused operations for efficiency
calibrated = temps * calibration                 # Broadcasting: (2,3) * (3,) -> (2,3)
weighted_avg = calibrated.t() @ weights          # Matrix multiply: (3,2) @ (2,) -> (3,)
print("Calibrated and weighted averages:", weighted_avg)

Calibrated and weighted averages: tensor([22.4910, 22.4616, 21.7453])


## Finding Patterns with SVD

SVD (Singular Value Decomposition) factorizes matrices into orthogonal components, enabling:
1. Dimensionality reduction with provable optimality
2. Noise filtering through low-rank approximation
3. Pattern discovery in high-dimensional data
    
![SVD Decomposition](figures/svd_decomposition.png)

Let's analyze how SVD helps with spam detection. First, let's look at some example emails:

In [13]:
# Example emails
emails = [
    """URGENT!! Make MONEY Fast! Buy our amazing product now!!!
    Don't miss this INCREDIBLE opportunity to earn $$$$$
    Click here: www.suspicious-link.com""",  # Spam
    
    """CONGRATULATIONS! You've WON $10,000,000!!!
    Send your bank details NOW to claim your PRIZE!!
    Visit: www.totally-legit-money.com/claim""",  # Spam
    
    """Dear valued customer, Your ACCOUNT needs verification!
    URGENT: Click here to prevent account suspension!!!
    www.bank-security-verify.com""",  # Spam
    
    """ACT NOW!!! Limited time offer - 90% OFF!!!
    Premium watches and luxury items at INCREDIBLE prices!
    Order here: www.discount-luxury-items.com""",  # Spam
    
    """ATTENTION: Your payment was declined!!!
    Update your billing information IMMEDIATELY!
    www.account-verify-now.com""",  # Spam
    
    """Hi team, Here's the quarterly report for Q3 2023.
    Please review the attached spreadsheet and let me know
    if you have any questions.""",  # Not spam
    
    """Meeting reminder: Project sync tomorrow at 10am.
    Agenda items: 1. Sprint review 2. Planning
    Please come prepared with your updates.""",  # Not spam
    
    """Thank you for your order #12345.
    Your package has been shipped and will arrive in 2-3 days.
    Track your delivery at: shipping.legitimate-store.com""",  # Not spam
    
    """Weekly team newsletter: 
    1. New hire welcome
    2. Office updates
    3. Upcoming events""",  # Not spam
    
    """Your library book is due in 3 days.
    Please return or renew online at library.edu/renew
    Thank you for using our services."""  # Not spam
]

# Feature extraction functions
def count_exclamations(text):
    """Count exclamation marks"""
    return text.count('!')

def count_urgent_words(text):
    """Count urgent words like 'urgent', 'now', 'act'"""
    urgent = ['urgent', 'now', 'act', 'immediate', 'fast']
    return sum(text.lower().count(word) for word in urgent)

def count_suspicious_links(text):
    """Count suspicious links (simplified)"""
    suspicious = ['.com/', 'click', 'www.']
    return sum(text.lower().count(marker) for marker in suspicious)

def compute_caps_ratio(text):
    """Compute ratio of uppercase to total letters"""
    letters = sum(c.isalpha() for c in text)
    if letters == 0:
        return 0
    caps = sum(c.isupper() for c in text)
    return caps / letters

def get_length(text):
    """Get text length"""
    return len(text)

# Extract features from emails
features = []
for email in emails:
    features.append([
        count_exclamations(email),
        count_urgent_words(email),
        count_suspicious_links(email),
        compute_caps_ratio(email),
        get_length(email)
    ])

# Convert to tensor
X = torch.tensor(features, dtype=torch.float)

# Analyze with SVD
U, S, V = torch.linalg.svd(X)
print("Singular values:", S)
print("\nEnergy per pattern:", 100 * S**2 / torch.sum(S**2), "%")

Singular values: tensor([4.3175e+02, 9.2474e+00, 1.7934e+00, 1.3037e+00, 1.2639e-01])

Energy per pattern: tensor([9.9951e+01, 4.5852e-02, 1.7245e-03, 9.1130e-04, 8.5654e-06]) %


The decomposition reveals:
1. Feature patterns (V): Which features occur together
2. Email patterns (U): How emails combine features
3. Pattern strengths (S): How important each pattern is

Looking at the first pattern:

In [14]:
print("Feature pattern:", V[0])
print("Email pattern:", U[:, 0])

# Look at second pattern for spam detection
print("\nSecond feature pattern:", V[1])
print("Second email pattern:", U[:, 1])

Feature pattern: tensor([-0.0206, -0.0076, -0.0061, -0.0010, -0.9997])
Email pattern: tensor([-0.3546, -0.3245, -0.3314, -0.3408, -0.2758, -0.3219, -0.3219, -0.3543,
        -0.2153, -0.2964])

Second feature pattern: tensor([ 0.9283,  0.2724,  0.2503,  0.0304, -0.0227])
Second email pattern: tensor([ 0.3693,  0.3426,  0.1339,  0.4278,  0.1956, -0.3123, -0.3417, -0.3762,
        -0.2285, -0.3147])


This dominant pattern shows:
1. Features: The first singular vector is dominated by text length (-0.9997), with negligible contributions from other features. When weighted by the large first singular value (431.75), this indicates that text length is the primary distinguishing feature.

2. Email patterns: The first left singular vector shows similar weights across all emails (around -0.3), indicating that text length alone does not effectively separate spam from non-spam. This makes sense - both spam and legitimate emails can be long or short.

3. Looking at the second singular value (9.2474) and its corresponding vectors:
   ```python
   print("Second feature pattern:", V[1])
   print("Second email pattern:", U[:, 1])
   ```
   This reveals that exclamation marks and urgent words are more discriminative. The second singular vector in V shows larger weights for these spam-indicative features, and the corresponding U vector more clearly separates spam (first 5 emails) from non-spam (last 5 emails).

Now let's analyze temperature patterns using PyTorch's highly optimized SVD implementation:

In [15]:
# Week of temperature readings
temps = torch.tensor([
    [22.5, 23.1, 21.8],  # Day 1: morning, noon, night
    [21.0, 22.5, 20.9],  # Day 2
    [23.1, 24.0, 22.8],  # Day 3
    [22.8, 23.5, 21.9],  # Day 4
    [21.5, 22.8, 21.2]   # Day 5
], dtype=torch.float)

# Compute SVD using optimized LAPACK routines
U, S, V = torch.linalg.svd(temps)
print("Singular values:", S)
print("\nEnergy per pattern:", 100 * S**2 / torch.sum(S**2), "%")
print("\nNumerical rank:", torch.linalg.matrix_rank(temps).item())

Singular values: tensor([86.6684,  0.5942,  0.2883])

Energy per pattern: tensor([9.9994e+01, 4.6999e-03, 1.1066e-03]) %

Numerical rank: 3


The decomposition reveals the temperature data's intrinsic dimensionality:
1. Feature patterns (V): Principal temperature variation modes
2. Day patterns (U): How each day combines these modes
3. Pattern strengths (S): Relative importance of each mode

The rapid decay of singular values indicates low intrinsic dimensionality - daily temperatures follow strong physical constraints.

In [16]:
# Analyze the dominant pattern
print("Time of day pattern (row 1 of V):", V[0])
print("Day pattern (column 1 of U):", U[:, 0])

# Compute percentage of variance explained
total_var = torch.sum(S**2)
explained_var = torch.cumsum(S**2, dim=0) / total_var
print("\nCumulative variance explained:", 100 * explained_var, "%")

Time of day pattern (row 1 of V): tensor([-0.5726, -0.5982, -0.5606])
Day pattern (column 1 of U): tensor([-0.4491, -0.4292, -0.4658, -0.4545, -0.4365])

Cumulative variance explained: tensor([ 99.9942,  99.9989, 100.0000]) %


### Pattern Analysis Example: Checkerboard
    
A synthetic example reveals how SVD decomposes structured patterns:

In [17]:
# Create checkerboard pattern with controlled noise
pattern = torch.tensor([
    [200,  50, 200,  50],
    [ 50, 200,  50, 200],
    [200,  50, 200,  50],
    [ 50, 200,  50, 200]
], dtype=torch.float)

# Add small random noise to test stability
noisy_pattern = pattern + torch.randn_like(pattern) * 0.1

# Analyze clean vs noisy patterns
U, S, V = torch.linalg.svd(noisy_pattern)
print("Singular values:", S)
print("\nEnergy per pattern:", 100 * S**2 / torch.sum(S**2), "%")
print("\nEffective numerical rank:", torch.sum(S > 1e-10).item())

Singular values: tensor([5.0012e+02, 3.0011e+02, 1.7744e-01, 4.7424e-02])

Energy per pattern: tensor([7.3525e+01, 2.6475e+01, 9.2553e-06, 6.6112e-07]) %

Effective numerical rank: 4


The SVD reveals the checkerboard's structure:
1. First component: Overall intensity (constant background)
2. Second component: Alternating pattern (checkerboard)
3. Remaining components: Numerical noise (~10⁻¹⁴)
    
This clean separation demonstrates SVD's power in pattern extraction.

In [18]:
# Reconstruct with different ranks using efficient matrix operations
def reconstruct(U, S, V, k):
    # Efficient reconstruction avoiding full matrix materialization
    return (U[:, :k] @ (torch.diag(S[:k]) @ V[:k, :]))

# Compare reconstructions
rank1 = reconstruct(U, S, V, 1)  # Background only
rank2 = reconstruct(U, S, V, 2)  # Full pattern

print("Original first row:", noisy_pattern[0])
print("Rank 1 first row:", rank1[0])
print("Rank 2 first row:", rank2[0])
print("\nReconstruction error:")
print(f"Rank 1: {torch.norm(noisy_pattern - rank1, p='fro'):.3f}")
print(f"Rank 2: {torch.norm(noisy_pattern - rank2, p='fro'):.3f}")

Original first row: tensor([199.9576,  50.0306, 199.9225,  50.0035])
Rank 1 first row: tensor([124.9520, 124.9886, 124.8831, 124.9990])
Rank 2 first row: tensor([200.0140,  50.0284, 199.8661,  50.0058])

Reconstruction error:
Rank 1: 300.106
Rank 2: 0.184


### Measuring Pattern Quality
    
The Frobenius norm provides a principled way to measure reconstruction quality:

In [19]:
# Three equivalent computations demonstrating numerical stability
print(f"Element-wise:    {torch.sqrt((pattern**2).sum()):.1f}")
print(f"As vector:       {pattern.view(-1).norm(p=2):.1f}")
print(f"From SVD:        {torch.norm(S, p=2):.1f}")

# Demonstrate optimality of SVD approximation
def random_rank2(shape):
    """Generate random rank-2 matrix"""
    return torch.randn(shape[0], 2) @ torch.randn(2, shape[1])

# Compare with random rank-2 approximation
random_approx = random_rank2(pattern.shape)
svd_approx = reconstruct(U, S, V, 2)

print(f"\nRandom rank-2 error: {torch.norm(pattern - random_approx, p='fro'):.1f}")
print(f"SVD rank-2 error:    {torch.norm(pattern - svd_approx, p='fro'):.1f}")

Element-wise:    583.1
As vector:       583.1
From SVD:        583.3

Random rank-2 error: 584.6
SVD rank-2 error:    0.4


### Summary
    
PyTorch implements linear algebra through three key mechanisms, each optimized for performance:

1. Tensors: Flexible N-dimensional arrays
   - Hardware-accelerated operations
   - Automatic memory management
   - Efficient data movement

2. Broadcasting: Implicit shape matching
   - Zero-copy operations
   - Cache-friendly access patterns
   - Automatic parallelization

3. SVD: Pattern discovery and compression
   - LAPACK-optimized implementation
   - Numerically stable algorithms
   - Automatic workspace management

In [20]:
# Essential operations summary with performance notes
x = torch.tensor([1, 2, 3], dtype=torch.float32)  # Contiguous memory allocation
y = torch.zeros_like(x)                           # Pre-allocated memory
z = torch.randn(3, 3)                            # Vectorized random generation
A = z                                            # Already float32 from randn

b = x + 2                                        # Fused operation
c = torch.dot(x, x)                             # BLAS optimized
d = A @ x                                       # Matrix multiply (GEMV)
e = torch.mean(A, dim=0)                        # Stable reduction

f = A.t()                                       # View only - no copy
g = A.view(-1)                                  # Reshape without copy
h = A[:, :2]                                    # Efficient slicing

U, S, V = torch.linalg.svd(A)                   # LAPACK optimized
norm = torch.norm(x, p=2)                       # Stable computation